# Playing Pong with Deep Reinforcement Learning

---

Read the paper [Playing Atari with Deep Reinforcement Learning](https://arxiv.org/pdf/1312.5602.pdf) (the paper is also inside the 'Papers' folder in the course materials), and implement a model that can play atari games.

The goals of this project are the following:

- Read and understand the paper.
- Add a brief summary of the paper at the start of the notebook.
- Mention and implement the preprocessing needed; you can add your own steps if needed.
- Load an Atari environment from OpenAI Gym; start with Pong, and try with at least one more.
- Define the convolutional model needed for training.
- Apply deep q learning with your model.
- Use the model to play a game and show the result.

**Rubric:**

1. A summary of the paper was included. The summary covered what the paper does, and why, as well as the preprocessing steps and the model they introduced.
2. Read images from the environment, and performed the correct preprocessing steps.
3. Defined an agent class with the needed functions.
4. Defined the model within the agent class.
5. Trained the model with the Pong environment. Save the weights after each episode.
6. Test the model by making it play Pong.
7. Train and test the agent with another Atari environment of your choosing.


## Add a summary of the paper in this cell

The paper talked about the posibility to create a neuronal network that can learn to play different games without any changes in the architecture of the network, the only change was the input that was a part of the screen of the game the network tried to learn.

They described how they calculated the reward and function implemented. Also mentioned that one movement in the game may take a lot of timesteps to see that movement result and give a proper reward, either negative or positive. There was a problem with the score too, different games means different ways to increase the score and therefore different increasing values, they opted to set positive rewars to one and negative to -1 to reduce complexity.

The images were preprocessed by pasing the image to gray scale, downsampling and corping it. This results in a square image. Then they merged four of those images (differente frames) and sended to the network, so the input was NxNx4. Also there was an skipping of four frames to speed up the training and it didn't affected the trainning to much, just in space invaders where the laser blinks.

The network they used was a small one, two convolutional layers, the flatten and two dense layers, one for analisis and the other for the result.

For their experiments they used a batch size of 32, RMSProp and a linear replay memory of 1 million.

### Basic installs and imports for Colab

In [1]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!pip install gym[box2d] > /dev/null 2>&1

    100% |████████████████████████████████| 583kB 27.3MB/s 
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: setuptools 40.9.0
    Uninstalling setuptools-40.9.0:
      Successfully uninstalled setuptools-40.9.0


In [2]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor

import matplotlib
import matplotlib.pyplot as plt

import cv2
import numpy as np
import random, math

from keras import models, layers, optimizers

from collections import deque

import glob, io, base64

from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display

gymlogger.set_level(40) #error only
%matplotlib inline

Using TensorFlow backend.


### Functions that wraps a video in colab

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [4]:
display = Display(visible=0, size=(1400, 900))
display.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [5]:
# Loads the pong environment
env = wrap_env(gym.make('Pong-v0'))

state_size = env.observation_space.shape[0]
action_size = env.action_space.n

print(state_size, action_size)

batch_size = 32

n_episodes = 1001

210 6


In [6]:
show_video()

Could not find video


## Define the Deep Q learning Agent

In [0]:
class DQNAgent:
    
    def __init__(self, state_size, action_size):
      
        self.state_size = state_size
        self.action_size = action_size
        
        self.memory = deque(maxlen=40000) #40k * 4 = 160k frames
        
        self.gamma = 0.999 #changed gamma from .95 to .999
        self.alpha = 0.001
        self.epsilon = 1.0
        self.epsilon_decay = 0.99997 
        self.epsilon_min = 0.01
        
        self.model = self._build_model()
        

    def _build_model(self):
        
        model = models.Sequential()
        model.add(layers.Conv2D(16, (8,8), strides=4, input_shape = (4,84,84), data_format="channels_first"))
        model.add(layers.LeakyReLU(alpha=0.3))
        model.add(layers.Conv2D(32, (4,4), strides=2))
        model.add(layers.LeakyReLU(alpha=0.3))
        model.add(layers.Flatten())
        model.add(layers.Dense(256, activation='relu'))
        model.add(layers.Dense(self.action_size, activation='softmax')) #movements
        
        return model
    
    def remember(self, state, action, reward, next_state, done, screen):
        '''
            state, action, reward at current time
            next_state is the state that occurs after the state-action
            done is if the episode ended
        '''
        self.memory.append((state, action, reward, next_state, done, screen))
        
    def action(self, screen):
        
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        
        prediction = self.model.predict(screen)
        #print("prediction: ",prediction)
        
        return np.argmax(prediction[0])
        
    def train(self, batch_size):
        
        batch = random.sample(self.memory, batch_size)
        
        for state, action, reward, next_state, done, screen in batch:
            
            target = reward
            
            if not done:
                
                target = (reward + self.gamma * 
                          np.amax(self.model.predict(screen)[0]))
                
            target_y = self.model.predict(screen)
            
            target_y[0][action] = target
            
            self.model.fit(screen, target_y, epochs=1, verbose=0)
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
           
    def load(self, name):
        self.model.load_weights(name)
        
    def save(self, name):
        self.model.save_weights(name)

In [8]:
agent = DQNAgent(state_size, action_size)
agent.model.summary()
agent.model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['accuracy'])


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 20, 20)        4112      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 20, 20)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 9, 32)          10272     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 7, 9, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2016)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               516352    
_________________________________________________________________
dens

### Needed preprocessing steps

In [0]:
def preprocessFrame(image): #210,160
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  image = cv2.resize(image,(84,110)) #resize
  image = image[18:102, :]  # crop the useless areas
  
  return image

In [10]:
"""#env.reset()
env.step(2)
img = env.render(mode='rgb_array') #get the screen
print(preprocessFrame(img).shape)

plt.imshow(preprocessFrame(img), cmap="gray")"""

'#env.reset()\nenv.step(2)\nimg = env.render(mode=\'rgb_array\') #get the screen\nprint(preprocessFrame(img).shape)\n\nplt.imshow(preprocessFrame(img), cmap="gray")'

## Training with the environment images
instead of perform an action every four frames keep the same for another three and change it in the fourth

In [15]:
#agent.load('12_10000_pong2_fixed2.hdf5')
#agent.epsilon =0.037770415418562506 
batch_size = 32
frames_pack = []
try:
    for e in range(10001): #epochs
        
        state = env.reset()
        
        total_reward = 0
        done = False
        action = 0 
        
        while not done:
                          
            next_state, reward, done, info = env.step(action)
            screen = env.render(mode='rgb_array') #get the screen
            screen = preprocessFrame(screen)
            frames_pack.append(screen) #stack images 
            screen = np.expand_dims(screen, axis=2)
            screen = np.expand_dims(screen, axis=0)
            
            if len(frames_pack) == 4: 
              frames_pack = np.array(frames_pack) # 4,84,84
              frames_pack = np.expand_dims(frames_pack, axis=0) #1,4,84,84

              # Define the reward for this problem
              reward = reward if not done else -1
              total_reward += reward

              agent.remember(state,action,reward,next_state,done,frames_pack)

              state = next_state


              # Takes a new action for the next 4 frames
              action = agent.action(frames_pack)        

              #Clear the frames stored
              frames_pack = []
            else:
              #safe data
              reward = reward if not done else -1
              total_reward += reward
              state = next_state
              
            
        
        if e % 100 == 0:
            agent.save('14_{:05d}_pong2_fixed2'.format(e) + '.hdf5')
            print(e, " : ",agent.epsilon, " : ",total_reward)
        if len(agent.memory) > batch_size:
          agent.train(batch_size)
                
        
finally:
    env.close()

0  :  0.027980046601426542  :  -21.0
100  :  0.027896230990660666  :  -21.0
200  :  0.027812666453694217  :  -21.0
300  :  0.027729352238423224  :  -21.0
400  :  0.027646287594996675  :  -21.0
500  :  0.02756347177580982  :  -21.0
600  :  0.027480904035497362  :  -21.0
700  :  0.027398583630926766  :  -21.0
800  :  0.027316509821191628  :  -21.0
900  :  0.02723468186760492  :  -20.0
1000  :  0.027153099033692455  :  -21.0
1100  :  0.027071760585186108  :  -21.0
1200  :  0.026990665790017358  :  -21.0
1300  :  0.02690981391831059  :  -21.0
1400  :  0.026829204242376593  :  -21.0
1500  :  0.02674883603670597  :  -21.0
1600  :  0.026668708577962642  :  -21.0
1700  :  0.02658882114497726  :  -21.0
1800  :  0.026509173018740877  :  -21.0
1900  :  0.026429763482398266  :  -21.0
2000  :  0.026350591821241644  :  -21.0
2100  :  0.026271657322704166  :  -21.0
2200  :  0.026192959276353466  :  -21.0
2300  :  0.026114496973885355  :  -21.0
2400  :  0.026036269709117387  :  -21.0
2500  :  0.025958

KeyboardInterrupt: ignored

In [13]:
#!rm *

!ls

12_00000_pong2_fixed2.hdf5  13_04900_pong2_fixed2.hdf5
12_00100_pong2_fixed2.hdf5  13_05000_pong2_fixed2.hdf5
12_00200_pong2_fixed2.hdf5  13_05100_pong2_fixed2.hdf5
12_00300_pong2_fixed2.hdf5  13_05200_pong2_fixed2.hdf5
12_00400_pong2_fixed2.hdf5  13_05300_pong2_fixed2.hdf5
12_10000_pong2_fixed2.hdf5  13_05400_pong2_fixed2.hdf5
13_00000_pong2_fixed2.hdf5  13_05500_pong2_fixed2.hdf5
13_00100_pong2_fixed2.hdf5  13_05600_pong2_fixed2.hdf5
13_00200_pong2_fixed2.hdf5  13_05700_pong2_fixed2.hdf5
13_00300_pong2_fixed2.hdf5  13_05800_pong2_fixed2.hdf5
13_00400_pong2_fixed2.hdf5  13_05900_pong2_fixed2.hdf5
13_00500_pong2_fixed2.hdf5  13_06000_pong2_fixed2.hdf5
13_00600_pong2_fixed2.hdf5  13_06100_pong2_fixed2.hdf5
13_00700_pong2_fixed2.hdf5  13_06200_pong2_fixed2.hdf5
13_00800_pong2_fixed2.hdf5  13_06300_pong2_fixed2.hdf5
13_00900_pong2_fixed2.hdf5  13_06400_pong2_fixed2.hdf5
13_01000_pong2_fixed2.hdf5  13_06500_pong2_fixed2.hdf5
13_01100_pong2_fixed2.hdf5  13_06600_pong2_fixed2.hdf5
13_01200_p

### Test your model

In [14]:
#1_10000  :  0.7408148869407416  :  -20.0
#2_10000  :  0.5487902325121208  :  -21.0
#3_6300  :  0.4542662331726328  :  -21.0
#4_5800  :  0.38171751014207483  :  -21.0
#5_10000  :  0.28278201411920023  :  -20.0
#6_10000  :  0.2094828411448149  :  -20.0
#7_10000  :  0.15518335163850305  :  -21.0
#8_7100  :  0.1254119773205814  :  -21.0
#9_10000  :  0.09290705979976088  :  -21.0
#10_10000  :  0.06882486819356542  :  -20.0
#11_10000  :  0.05098648694952733  :  -21.0
#12_10000  :  0.037770415418562506  :  -21.0
#13_10000  :  0.02798088602800738  :  -21.0
#14_8400  :  0.021747261785438208  :  -21.0
#agent.load('8_9900_pong2_fixed.hdf5')
print(agent.epsilon)
env = wrap_env(gym.make('Pong-v0'))
frames_pack= []

try:
      action = 0 #do nothing in context of pong

      state = env.reset()

      total_reward = 0
      done = False

      while not done:
      #for time in range(200):

          #env.render()

          # Makes an action, screen capture and preprocess
          next_state, reward, done, info = env.step(action)
          screen = env.render(mode='rgb_array') #get the screen
          screen = preprocessFrame(screen) #84x84
          frames_pack.append(screen) #stack images 
          
          if len(frames_pack) == 4: #all the images for a single input
            
            frames_pack = np.array(frames_pack) # 4,84,84
            frames_pack = np.expand_dims(frames_pack, axis=0) #1,4,84,84
            
            # Define the reward for this problem
            reward = reward if not done else -1
            total_reward += reward
            
            agent.remember(state,action,reward,next_state,done,frames_pack)
            
            state = next_state


            # Takes a new action for the next 4 frames
            action = agent.action(frames_pack)        
            
            #Clear the frames stored
            frames_pack = []
            
            
          else:
            #safe data
            reward = reward if not done else -1
            total_reward += reward
            state = next_state
        
finally:
    env.close()       
    show_video()
    print(total_reward)

0.027980046601426542


-21.0


## Train and test your agent with another atari environment

In [0]:
# Loads the pong environment
env = wrap_env(gym.make('Breakout-v0'))

state_size = env.observation_space.shape[0]
action_size = env.action_space.n

print(state_size, action_size)

In [0]:
agent2 = DQNAgent(state_size, action_size)
agent2.model.summary()
agent2.model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['accuracy'])


###Train  another enviroment###

In [0]:
#agent2.load('11_10000_breakout.hdf5') #if you load weights you must also define the epsilon value
#agent2.epsilon = 0.041081479346882256
batch_size = 32
frames_pack = []
try:
    for e in range(10001): #epochs
        
        state = env.reset()
        
        total_reward = 0
        done = False
        action = 0 
        
        while not done:
                          
            next_state, reward, done, info = env.step(action)
            screen = env.render(mode='rgb_array') #get the screen
            screen = preprocessFrame(screen)
            frames_pack.append(screen) #stack images 
            screen = np.expand_dims(screen, axis=2)
            screen = np.expand_dims(screen, axis=0)
            
            if len(frames_pack) == 4: 
              frames_pack = np.array(frames_pack) # 4,84,84
              frames_pack = np.expand_dims(frames_pack, axis=0) #1,4,84,84

              # Define the reward for this problem
              reward = reward if not done else -1
              total_reward += reward

              agent2.remember(state,action,reward,next_state,done,frames_pack)

              state = next_state


              # Takes a new action for the next 4 frames
              action = agent2.action(frames_pack)        

              #Clear the frames stored
              frames_pack = []
            else:
              #safe data
              reward = reward if not done else -1
              total_reward += reward
              state = next_state
              
            
        
        if e % 100 == 0:
            agent2.save('12_{:05d}_breakout'.format(e) + '.hdf5')
            print(e, " : ",agent2.epsilon, " : ",total_reward)
        if len(agent2.memory) > batch_size:
          agent2.train(batch_size)
                
        
finally:
    env.close()

###Test another enviroment###

In [0]:
#1_10000  :  0.7408148869407416  :  2.0
#2_10000  0.5487737688051454 
#3_10000  :  0.4065397774934281  :  0.0
#4_10000  :  0.30116168417912753  :  0.0
#5_10000  :  0.2230983658642711  :  0.0
#6_10000  :  0.1652745906844041  :  0.0
#7_10000  :  0.1224378772120437  :  0.0
#8_10000  :  0.09070108105003913  :  1.0
#9_6400  :  0.07485600958029996  :  1.0
#10_10000  :  0.05545444627406444  :  1.0
#11_10000  :  0.041081479346882256  :  -1.0
#12_04500  :  0.0358934694952554  :  -1.0
print(agent2.epsilon)
env = wrap_env(gym.make('Breakout-v0'))
frames_pack= []

try:
      action = 0 #do nothing in context of pong

      state = env.reset()

      total_reward = 0
      done = False

      while not done:

          # Makes an action, screen capture and preprocess
          next_state, reward, done, info = env.step(action)
          screen = env.render(mode='rgb_array') #get the screen
          screen = preprocessFrame(screen) #84x84
          frames_pack.append(screen) #stack images 
          
          if len(frames_pack) == 4: #all the images for a single input
            
            frames_pack = np.array(frames_pack) # 4,84,84
            frames_pack = np.expand_dims(frames_pack, axis=0) #1,4,84,84
            
            # Define the reward for this problem
            reward = reward if not done else -1
            total_reward += reward
            
            agent2.remember(state,action,reward,next_state,done,frames_pack)
            
            state = next_state


            # Takes a new action for the next 4 frames
            action = agent2.action(frames_pack)        
            
            #Clear the frames stored
            frames_pack = []
            
            
          else:
            #safe data
            reward = reward if not done else -1
            total_reward += reward
            state = next_state
        
finally:
    env.close()       
    show_video()
    print(total_reward)